In [1]:
# Imports
from os.path import join, isdir, isfile
from os import listdir as ls
import copy
import os
from IPython.display import display
from ipywidgets import interact, widgets
from PIL import Image
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import numpy as np

In [ ]:
# Local clone
! git clone https://github.com/nanopiero/weow.git
import sys
sys.path.append('weow')
from src.utils import *

In [ ]:
# To get models and images
! pip install huggingface_hub
from huggingface_hub import hf_hub_download

In [ ]:
import os
import zipfile
from huggingface_hub import hf_hub_download

# Step 1: Download the zip file using hf_hub_download
local_zip = hf_hub_download(repo_id="nanopiero/weow", filename="webcam_images.zip")

# Step 2: Unzip the contents
with zipfile.ZipFile(local_zip, 'r') as zip_ref:
    zip_ref.extractall('webcam_images')

In [ ]:
# List all .jpg images in the directory
image_dir = 'webcam_images'
image_files = sorted([f for f in os.listdir(image_dir) if f.endswith('.jpg')], key=lambda x: int(x.split('.')[0]))

def show_image(index):
    img_path = os.path.join('webcam_images', image_files[index])
    img = Image.open(img_path)
    plt.imshow(img)
    plt.axis('off')  # Hide axes
    plt.show()

# Use a slider to scroll through the images
interact(show_image, index=widgets.IntSlider(min=0, max=len(image_files)-1, step=1, description="Image Index"));


## Sorting with respect to Horizontal Visibility

In [ ]:
# Step 1: Download and Load the Models from Hugging Face

# List of models to download and use
model_names = [
        'vv_sl_2024_fss_1.checkpoint',
        'vv_sl_2024_fss_2.checkpoint',
        'vv_sl_2024_iss_1.checkpoint' ,
        'vv_sl_2024_iss_2.checkpoint',
]

# Step 2: Initialize the Transformation and Load the Dataset

# Configuration for Simple_crop transformation
marginsup = 0.0
margininf = 0.0
cropped_prop = 1.0
size_in = 256 + 32
size_out = 256 - 32

# Instantiate the transformation
tr = Simple_crop(marginsup, margininf, cropped_prop, size_in, size_out)

# Create dataset and dataloader
image_dir = 'webcam_images'
dataset = WebcamImagesDataset(image_dir=image_dir, transform=tr)
dataloader = DataLoader(dataset, batch_size=1, shuffle=False)

# Step 3: Load Models onto GPU

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
models = [load_model(model_name) for model_name in model_names]

# Step 4: Perform Inference
mean_outputs = []

for images in dataloader:
    images = images.to(device)
    outputs = []

    # Run the image through each model and collect outputs
    for k, model in enumerate(models):
        with torch.no_grad():
            output = model(images)[0]
            if 'rev' in model_names[k]:
              output *= -1
            outputs.append(output)

    # Compute the mean of the outputs across models
    mean_output = torch.mean(torch.stack(outputs), dim=0)
    mean_outputs.append(mean_output.cpu().numpy())


## Sorting with respect to snow cover

In [ ]:
# Step 1: Download and Load the Models from Hugging Face

# List of models to download and use
model_names = [
        'ss_sl_rev_due222.0.pth',
        'ss_sl_rev_due222.1.pth',
        'ss_sl_due222.0.pth' ,
        'ss_sl_du22.0.pth',
        'ss_sl_due222.1.pth',
]

# Step 2: Initialize the Transformation and Load the Dataset

# Configuration for Simple_crop transformation
marginsup = 0.0
margininf = 0.0
cropped_prop = 1.0
size_in = 256 + 32
size_out = 256 - 32

# Instantiate the transformation
tr = Simple_crop(marginsup, margininf, cropped_prop, size_in, size_out)

# Create dataset and dataloader
image_dir = 'webcam_images'
dataset = WebcamImagesDataset(image_dir=image_dir, transform=tr)
dataloader = DataLoader(dataset, batch_size=1, shuffle=False)

# Step 3: Load Models onto GPU

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
models = [load_model(model_name) for model_name in model_names]

# Step 4: Perform Inference
mean_outputs = []

for images in dataloader:
    images = images.to(device)
    outputs = []

    # Run the image through each model and collect outputs
    for k, model in enumerate(models):
        with torch.no_grad():
            output = model(images)[0]
            if 'rev' in model_names[k]:
              output *= -1
            outputs.append(output)

    # Compute the mean of the outputs across models
    mean_output = torch.mean(torch.stack(outputs), dim=0)
    mean_outputs.append(mean_output.cpu().numpy())


## Visualization

In [ ]:
# To plot output interval
from google.colab import output
output.enable_custom_widget_manager()
!pip install bqplot
import bqplot as bq
from ipywidgets import HBox, Output, interact, IntSlider

def display_image(image_path):
    with output:
        output.clear_output(wait=True)  # Clear previous output
        img = Image.open(image_path)  # Load the image
        img = img.resize((int(img.width * 0.5), int(img.height * 0.5)))  # Resize the image by 50%
        display(img)  # Display the resized image using IPython.display

def show_image_from_index(index):
    # Create a new color list and update all intervals
    new_colors = ['gray'] * len(images)  # Reset all intervals to gray
    new_colors[index] = 'red'  # Highlight the selected interval in red
    lines.colors = new_colors  # Assign the new color list to the lines.colors trait

    # Display the corresponding image
    image_path = images[index]
    display_image(image_path)

In [ ]:
intervals = mean_outputs
image_files = sorted([ f for f in os.listdir(image_dir) \
                      if f.endswith('.jpg')], key=lambda x: int(x.split('.')[0]))

images = [join(image_dir, f) for f in image_files]
x_vals = list(range(len(images)))  # Corresponding indices for the intervals

# Convert intervals using .item() to extract the values
y_vals = [[interval[0, 0].item(), interval[0, 1].item()] for interval in intervals]

# Step 1: Create the Output widget to display the image
output = Output()

# Step 2: Create the bqplot figure
x_scale = bq.LinearScale()  # Change to LinearScale to spread the intervals evenly
y_scale = bq.LinearScale()

# Create the interval lines using the converted y_vals
lines = bq.Lines(
    x=[[x, x] for x in range(len(intervals))],  # X values are spread linearly
    y=y_vals,  # Use the converted y values
    scales={'x': x_scale, 'y': y_scale},
    stroke_width=4,  # Narrower intervals (reduce width)
    colors=['steelblue'] * len(intervals)  # Initial color for all intervals
)

# Step 3: Create the figure
# We display one x-label for every 10 intervals and make them smaller to avoid overlap
ax_x = bq.Axis(scale=x_scale, label='Index', tick_values=list(range(0, 211, 10)), tick_style={'font-size': 8})
ax_y = bq.Axis(scale=y_scale, orientation='vertical', label='Interval')

# Reduce the height of the plot by 40%
fig = bq.Figure(marks=[lines], axes=[ax_x, ax_y], title='Intervals with Slider Control',
                fig_margin={'top': 60, 'bottom': 50, 'left': 60, 'right': 50},
                layout={'min_width': '1000px', 'min_height': '100px'})  # Reduce height by 40%

# Step 4: Create the slider and connect it to the `show_image` function

slider = IntSlider(
    min=0,
    max=209,  # Maximum value to match your x-axis
    step=1,  # Step of 10 for the graduations
    description="Image Index",
    layout={'width': '1000px'},  # Set slider width (length)
    readout_format='d'  # Display integer format for readout
)

interact(show_image_from_index, index=slider)

# Step 5: Display the HBox with the bqplot figure and Output widget side by side
hbox = HBox([fig, output], layout={'width': '100%'})  # Set the layout to take full width
display(hbox)
